# Deep Learning Satellite Image Classification Pipeline

**Project Steps:**
1. Data Loading & Preprocessing (Resize to 224x224)
2. Train 3 DenseNet Models (121, 169, 201)
3. Select Top 2 Best Performing Models
4. Feature Extraction & Fusion (Dimension Reduction)
5. Train Classical Classifiers (RF, DT, Softmax) on Fused Features
6. Select Best Classifier & Evaluate

## Step 1: Configuration & Imports

In [ ]:
import os
import json
import cv2
import numpy as np
import tensorflow as tf
import operator
import matplotlib.pyplot as plt
from tensorflow.keras.applications import DenseNet121, DenseNet169, DenseNet201
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.backend as K
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

TRAIN_PATH = r"C:\Users\Michael\Desktop\Research Capstone"
CLASSES = ['Lake', 'Desert', 'Mountain']
IMG_SIZE = 224
BATCH_SIZE = 8
EPOCHS = 40
LEARNING_RATE = 1e-4

## Step 2: Data Processing

In [ ]:
print("--- Loading & Resizing Images ---")

X_train_list, y_train_list = [], []
X_test_list, y_test_list = [], []

for label_id, label_name in enumerate(CLASSES):
    folder_path = os.path.join(TRAIN_PATH, label_name)
    if not os.path.exists(folder_path):
        print(f"Warning: {folder_path} not found.")
        continue

    image_files = os.listdir(folder_path)[:1200]
    print(f"Processing {label_name}: {len(image_files)} images.")

    class_imgs = []
    for file in image_files:
        try:
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                class_imgs.append(img)
        except:
            pass

    class_imgs = np.array(class_imgs)

    X_train_list.extend(class_imgs[:800])
    y_train_list.extend([label_id] * 800)
    X_test_list.extend(class_imgs[800:920])
    y_test_list.extend([label_id] * 120)

X_train = np.array(X_train_list).astype('float32') / 255.0
y_train = np.array(y_train_list)
X_test = np.array(X_test_list).astype('float32') / 255.0
y_test = np.array(y_test_list)

print(f"Data Ready: Train {X_train.shape}, Test {X_test.shape}")

## Step 3: Train DenseNet Models & Select Top 2

In [ ]:
densenet_variants = {
    'DenseNet121': DenseNet121,
    'DenseNet169': DenseNet169,
    'DenseNet201': DenseNet201
}

trained_models = {}
model_accuracies = {}
model_histories = {}   # <--- new dict to store histories

for model_name, model_func in densenet_variants.items():
    print(f"Training {model_name}...")
    K.clear_session()
    base_model = model_func(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    x = GlobalAveragePooling2D()(base_model.output)
    predictions = Dense(3, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=SGD(learning_rate=LEARNING_RATE, momentum=0.9),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_test, y_test)
    )
    loss, acc = model.evaluate(X_test, y_test)
    trained_models[model_name] = model
    model_accuracies[model_name] = acc
    model.save(f"{model_name}.h5")
    with open(f"{model_name}_history.json", "w") as f:
        json.dump(history.history, f)

sorted_models = sorted(model_accuracies.items(), key=lambda x: x[1], reverse=True)
best_name_1, best_name_2 = sorted_models[0][0], sorted_models[1][0]
model_A, model_B = trained_models[best_name_1], trained_models[best_name_2]





## Training Loss and Validation Graph DenseNet201

In [ ]:
# Training loss and validation loss graph (SMOOTH)

epochs = range(len(history.history['loss']))


plt.plot(epochs, history.history['loss'],
         color='gold', linewidth=2, label='Training Loss')

plt.plot(epochs, history.history['val_loss'],
         color='green', linewidth=2, label='Validation Loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.grid(False)

plt.savefig("Loss_Smooth.tiff", format="tiff", dpi=300, bbox_inches='tight')
plt.show()


## Training Accuracy and Validation Accuracy Graph DenseNet201

In [ ]:
# Training accuracy and validation accuracy graph (SMOOTH)

epochs = range(len(history.history['accuracy']))

plt.plot(epochs, history.history['accuracy'],
         color='gold', linewidth=2, label='Training Accuracy')

plt.plot(epochs, history.history['val_accuracy'],
         color='green', linewidth=2, label='Validation Accuracy')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.grid(False)  # clean look with no grid lines

plt.savefig("Accuracy_Smooth.tiff", format="tiff", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import numpy as np

# use the best model (from your sorted_models)
best_model = model_A        # this is what you already defined
# or: best_model = trained_models[best_name_1]

# predicted probabilities on test set
y_prob = best_model.predict(X_test)

# convert to class indices (0, 1, 2)
y_pred = np.argmax(y_prob, axis=1)


In [ ]:

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of best model: {:.3f}".format(accuracy))

In [ ]:
print("Best model:", best_name_1)
print("Best model:", best_name_2)



## DenseNet121

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

f = sns.heatmap(cm, annot=True, fmt='d', cmap="hot")

f.set_xlabel('Predicted label')
f.set_ylabel('True label')

f.xaxis.set_ticklabels(['Desert', 'Mountain', 'Lake'])
f.yaxis.set_ticklabels(['Desert', 'Mountain', 'Lake'])

plt.savefig("CM.tiff", format="tiff")
plt.show()

In [ ]:
import numpy as np
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

# Use DenseNet121 ONLY
dn121_model = trained_models["DenseNet121"]

# number of classes
classes = [0, 1, 2]

# predicted probabilities from DN121
y_prob = dn121_model.predict(X_test)

# binarize true labels
y_test_bin = label_binarize(y_test, classes=classes)

# compute micro-average ROC
fpr, tpr, _ = roc_curve(y_test_bin.ravel(), y_prob.ravel())
roc_auc = auc(fpr, tpr)

print(f"AUC (DenseNet121): {roc_auc:.3f}")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5,5))

plt.plot(fpr, tpr, label=f"DenseNet121 ROC (AUC = {roc_auc:.2f})", linewidth=2)

# Diagonal baseline
plt.plot([0, 1], [0, 1], 'y--', linewidth=1)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")

# Save
plt.savefig("ROC_DN121.tiff", format="tiff", dpi=300, bbox_inches='tight')
plt.show()


## Convolutional Layer


In [ ]:
best_model = model_A      # BEST model chosen by accuracy
img = X_test[0:1]         # any test image


In [ ]:
###### CL1 Feature Maps
M_CL1 = Model(inputs=best_model.inputs, outputs=best_model.layers[2].output)
feature_maps = M_CL1.predict(img)

from matplotlib import pyplot
square=5
ix=1
ax=pyplot.figure(figsize=(5,5))

for i in range(square):
    for j in range(square):
        ax = pyplot.subplot(square, square, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_aspect('equal')
        pyplot.imshow(feature_maps[0, :, :, ix-1], aspect='auto', cmap="hot")
        ix += 1

pyplot.savefig("CL1.tiff", format="tiff")
pyplot.show()


In [ ]:
###### CL2 Feature Maps
M_CL2 = Model(inputs=best_model.inputs, outputs=best_model.layers[8].output)
feature_maps = M_CL2.predict(img)

from matplotlib import pyplot
square=5
ix=1
ax=pyplot.figure(figsize=(5,5))

for i in range(square):
    for j in range(square):
        ax = pyplot.subplot(square, square, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_aspect('equal')
        pyplot.imshow(feature_maps[0, :, :, ix-1], aspect='auto', cmap="hot")
        ix += 1

pyplot.savefig("CL2.tiff", format="tiff")
pyplot.show()


In [ ]:
###### CL3 Feature Maps
M_CL3 = Model(inputs=best_model.inputs, outputs=best_model.layers[15].output)
feature_maps = M_CL3.predict(img)

from matplotlib import pyplot
square=5
ix=1
ax=pyplot.figure(figsize=(5,5))

for i in range(square):
    for j in range(square):
        ax = pyplot.subplot(square, square, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_aspect('equal')
        pyplot.imshow(feature_maps[0, :, :, ix-1], aspect='auto', cmap="hot")
        ix += 1

pyplot.savefig("CL3.tiff", format="tiff")
pyplot.show()


In [ ]:
###### CL4 Feature Maps
M_CL4 = Model(inputs=best_model.inputs, outputs=best_model.layers[25].output)
feature_maps = M_CL4.predict(img)

from matplotlib import pyplot
square=5
ix=1
ax=pyplot.figure(figsize=(5,5))

for i in range(square):
    for j in range(square):
        ax = pyplot.subplot(square, square, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_aspect('equal')
        pyplot.imshow(feature_maps[0, :, :, ix-1], aspect='auto', cmap="hot")
        ix += 1

pyplot.savefig("CL4.tiff", format="tiff")
pyplot.show()


## Step 4: Feature Fusion Strategy

In [ ]:
extractor_A = Model(inputs=model_A.input, outputs=model_A.layers[-2].output)
extractor_B = Model(inputs=model_B.input, outputs=model_B.layers[-2].output)

input_img = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
feat_a = extractor_A(input_img)
feat_b = extractor_B(input_img)
reduced_a = Dense(500, activation='relu')(feat_a)
reduced_b = Dense(500, activation='relu')(feat_b)
fused_vector = Concatenate()([reduced_a, reduced_b])
fused_model = Model(inputs=input_img, outputs=fused_vector)


## Step 5: Generate Fused Feature Vectors

In [ ]:
X_train_FFV = fused_model.predict(X_train)
X_test_FFV = fused_model.predict(X_test)

## Step 6: Classical Classification & Final Results

In [ ]:
classifiers = {
    'SoftMax (Logistic Regression)': LogisticRegression(max_iter=1000, multi_class='multinomial'),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators=100)
}

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
results = {}
for name, clf in classifiers.items():
    scores = cross_val_score(clf, X_train_FFV, y_train, cv=kfold)
    results[name] = scores.mean()

best_clf_name = max(results, key=results.get)
best_clf = classifiers[best_clf_name]
best_clf.fit(X_train_FFV, y_train)
pred = best_clf.predict(X_test_FFV)
print(classification_report(y_test, pred, target_names=CLASSES))

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

def compute_metrics(y_true, y_pred, num_classes=3):
    cm = confusion_matrix(y_true, y_pred, labels=np.arange(num_classes))

    # per-class TP,FN,TN,FP (one-vs-rest)
    TP_list = []
    FN_list = []
    TN_list = []
    FP_list = []

    for c in range(num_classes):
        TP = cm[c,c]
        FN = cm[c,:].sum() - TP
        FP = cm[:,c].sum() - TP
        TN = cm.sum() - (TP + FN + FP)

        TP_list.append(TP)
        FN_list.append(FN)
        TN_list.append(TN)
        FP_list.append(FP)

    # macro-average like article
    TP_macro = np.mean(TP_list)
    FN_macro = np.mean(FN_list)
    TN_macro = np.mean(TN_list)
    FP_macro = np.mean(FP_list)

    AC = (TP_macro + TN_macro) / (TP_macro + TN_macro + FP_macro + FN_macro)
    PR = TP_macro / (TP_macro + FP_macro)
    SE = TP_macro / (TP_macro + FN_macro)
    SP = TN_macro / (TN_macro + FP_macro)

    return TP_macro, FN_macro, TN_macro, FP_macro, AC, PR, SE, SP


In [ ]:
dn_results = {}

for name in ["DenseNet121", "DenseNet169", "DenseNet201"]:
    print(f"Evaluating {name}...")

    model = trained_models[name]

    y_prob = model.predict(X_test)
    y_pred = np.argmax(y_prob, axis=1)

    TP, FN, TN, FP, AC, PR, SE, SP = compute_metrics(y_test, y_pred)

    dn_results[name] = [TP, FN, TN, FP, AC*100, PR*100, SE*100, SP*100]


In [ ]:
fusion_results = {}

fusion_models = {
    "FFV-SM": classifiers['SoftMax (Logistic Regression)'],
    "FFV-DT": classifiers['Decision Tree'],
    "FFV-RF": classifiers['Random Forest']
}

for name, clf in fusion_models.items():
    print(f"Evaluating {name}...")

    clf.fit(X_train_FFV, y_train)
    pred = clf.predict(X_test_FFV)

    TP, FN, TN, FP, AC, PR, SE, SP = compute_metrics(y_test, pred)

    fusion_results[name] = [TP, FN, TN, FP, AC*100, PR*100, SE*100, SP*100]


In [ ]:
import pandas as pd

rows = []

# DenseNets
for name, vals in dn_results.items():
    rows.append([name] + list(np.round(vals, 4)))

# Fusion models
for name, vals in fusion_results.items():
    rows.append([name] + list(np.round(vals, 4)))

df = pd.DataFrame(rows, columns=[
    "Model", "TP", "FN", "TN", "FP", "AC", "PR", "SE", "SP"
])

df


In [ ]:
import matplotlib.pyplot as plt

# ---- Convert your existing df into an image ----
fig, ax = plt.subplots(figsize=(10, 3))

ax.axis('off')   # Hide axis

table = ax.table(
    cellText=df.values,
    colLabels=df.columns,
    loc='center',
    cellLoc='center'
)

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)

plt.savefig("Performance_Table.tiff", dpi=300, format="tiff", bbox_inches='tight')
plt.show()
